In [249]:
import scipy.signal as ss
import numpy as np

In [250]:
X = np.array([[1, 3, 5],
               [2, 7, 9],
               [11, 13, 17]], dtype='f8')
W = np.array([[19, 23],
              [29, 31]], dtype='f8')
# X = np.ones(12, dtype='f8').reshape(3,4) + 1
# W = np.ones(6, dtype='f8').reshape(2,3) + 1

$\newcommand{\bbR}{\mathbb{R}}$
A 2D convolution operation between the finite dimensional kernel weights $W \in \bbR^{K \times L}$ and image $X\in\bbR^{I \times J}$ is denoted as $Y = X * W\in \bbR^{I-K+1,J-L+1}$. Let the convolution kernel be smaller than the image in both the dimensions, $K < I$ and $L<J$.
Then the 2D convolution operation for all $i \in \{1, \dots, I-K+1\}, j\in \{1, \dots, J-K+1\}$ is  defined as,
\begin{align}
Y &= W * X\\
 Y_{i,j} &= \sum_{k=1}^K \sum_{l=1}^L W_{k,l} X_{i+K-k,j+L-l} \\
 &= \sum_{p=i}^{K+i-1} \sum_{q=j}^{L+j-1} X_{p, q} W_{i+K-p,j+L-q}
\end{align}
The two expressions are equivalent and can obtained by variable substitution of $p=i+K-k$ and $q=j+L-l$.

In [251]:
def handle_padding(padding):
  if padding is None:
    padding = [(0, s-1) for s in W.shape]
  return padding
def t_convolve2d(X, W, padding=None):
  padding = handle_padding(padding)
  Xpadded = np.pad(X, padding)
  Y = ss.convolve2d(Xpadded, W, mode='valid')
  return Y

Y = t_convolve2d(X, W)
Y, (X[:2, :2] * W).sum(), (X[:2, :2] * W[::-1, ::-1]).sum()

(array([[ 297.,  570.,  362.],
        [ 765., 1100.,  670.],
        [ 718.,  896.,  527.]]),
 363.0,
 297.0)

## Finding vector jacobian product
$\newcommand{\p }{\partial}$
$\newcommand{\pfrac}[2]{\frac{\p {#1}}{\p {#2}}}$
Let the eventual loss function be $l(Y)$ and we are given the backpropagated gradient $\pfrac{l}{Y}$. We want to find the vector jacobian products, $\pfrac{l}{X}$ and $\pfrac{l}{W}$.

\begin{align}
\pfrac{}{W} l(Y) &= \sum_{i=1}^{I-K+1} \sum_{j=1}^{J-L+1} \pfrac{l}{Y_{i,j}} \pfrac{Y_{i,j}}{W}\\
\pfrac{}{X} l(Y) &= \sum_{i=1}^{I-K+1} \sum_{j=1}^{J-L+1} \pfrac{l}{Y_{i,j}} \pfrac{Y_{i,j}}{X}
\end{align}

### Finding $\pfrac{l}{W}$
Our main task is to find $\pfrac{Y_{i,j}}{W}$. Let's find the $(k,l)$th element of this derivative,

\begin{align}
\pfrac{Y_{i,j}}{W_{k,l}}= \pfrac{}{W_{k,l}} \sum_{k=1}^K \sum_{l=1}^L W_{k,l} X_{i+K-k,j+L-l} &=
X_{i+K-k,j+L-l} &\text{ for all} i \le I, k \le K, j \le J, l \le L
\end{align}

That was easy. Let's put it back in the equation one above to find $\pfrac{l}{W_{k,l}}$
\begin{align}
\pfrac{}{W_{k,l}} l(Y) &= \sum_{i=1}^{I-K+1} \sum_{j=1}^{J-L+1} \pfrac{l}{Y_{i,j}}X_{i+K-k,j+L-l}.
\end{align}

$\newcommand{\calM}{\mathcal{M}}$
Define mirroring operator of an image $Z \in \bbR^{I \times J}$ as $\calM(Z)$ as,
\begin{align}
\calM(Z)_{i,j} &= Z_{I+1-i,J+1-j}
\end{align}
Rewrite $\pfrac{l}{W_{k,l}}$ in terms of mirrored $\calM(\pfrac{l}{W})_{m,n}$,
\begin{align}
\calM\left(\pfrac{}{W} l(Y)\right)_{m,n}
&= \sum_{i=1}^{I-K+1} \sum_{j=1}^{J-L+1} \pfrac{l}{Y_{i,j}}X_{i+K-(K+1-m),j+L-(L+1-n)}\\
&= \sum_{i=1}^{I-K+1} \sum_{j=1}^{J-L+1} \pfrac{l}{Y_{i,j}}X_{m-1+i,n-1+j}\\
\end{align}
Change the indexing variables $p=I-K+2-i$ and $q=J-L+2-j$,
\begin{align}
\calM\left(\pfrac{}{W} l(Y)\right)_{m,n}
 &= \sum_{p=1}^{I-K+1} \sum_{q=1}^{J-L+1} \calM\left(\pfrac{l}{Y}\right)_{p,q} X_{m+I-K+1-p,n+J-L+1-n}.
\end{align}
Left hand side of the above equation is a convolution operation, with $\pfrac{l}{Y}$ as the kernel,
\begin{align}
\calM\left(\pfrac{}{W} l(Y)\right) &=  \calM(\pfrac{l}{Y}) * X\\
\implies \pfrac{}{W} l(Y) &= \calM\left( \calM(\pfrac{l}{Y}) * X \right)
\end{align}

It turns out the following is equivalent,
\begin{align}
\pfrac{}{W} l(Y) &= \pfrac{l}{Y} * \calM(X)
\end{align}

### Finding $\pfrac{l}{X}$
Let's try to do the same with $\pfrac{l}{X}$ by finding $\pfrac{Y_{i,j}}{X_{p,q}}$ first,
\begin{align}
\pfrac{Y_{i,j}}{X_{p,q}} &= \pfrac{}{X_{p,q}} \sum_{p=i}^{K+i-1} \sum_{q=j}^{L+j-1} X_{p, q} W_{i+K-p,j+L-q}\\
&= \begin{cases}
W_{i+K-p,j+L-q} & \text{ if } 1 \le i+K-p \le K \text { and } 1 \le j+L-q \le J\\
0 & \text{ otherwise }
\end{cases}.
\end{align}


Using this we can find the $(p,q)$th element of $\pfrac{l}{X}$ as,
\begin{align}
\pfrac{}{X_{p,q}} l(Y) &= \sum_{i=1}^{I-K+1} \sum_{j=1}^{J-L+1} \pfrac{l}{Y_{i,j}} \pfrac{Y_{i,j}}{X_{p,q}}\\
&= \sum_{i=1}^{p-K} \sum_{j=1}^{p-L} \pfrac{l}{Y_{i,j}} . 0
+ \sum_{i=p-K+1}^{p} \sum_{j=q-L+1}^{q} \pfrac{l}{Y_{i,j}}  W_{i+K-p,j+L-q}\\
&\qquad + \sum_{i=p+1}^{I-K+1} \sum_{j=q+1}^{J-L+1} \pfrac{l}{Y_{i,j}} . 0\\
&= \sum_{i=p-K+1}^{p} \sum_{j=q-L+1}^{q} \pfrac{l}{Y_{i,j}}  W_{i+K-p,j+L-q}.
\end{align}
In the last step we reduced the scope  of summation where $\pfrac{Y_{i,j}}{X_{p,q}}$ is non-zero. However, due to the change of variables, some of the indexing on $\pfrac{l}{Y_{i,j}}$ is invalid. To remedy that define a padded gradient $G \in \bbR^{I+K-1 \times J+L-1}$ that is defined as,
\begin{align}
G_{m, n} &= \begin{cases}
0 & \text{ if } n \le K-1 \text{ or } m \le L-1 \\
\pfrac{l}{Y_{m-K+1,n-L+1}} & \text{ if } K \le m \le I \text{ and } L \le n \le J\\
0 & \text{ if } I+1 \le n \le I+K-1 \text{ or } J+1 \le m \le J+L-1 \\
\end{cases}\\
\implies \pfrac{l}{Y_{i,j}} &= G_{i+K-1,j+L-1}
\end{align}
Put this back into $\pfrac{l}{X_{p,q}}$,
\begin{align}
\pfrac{}{X_{p,q}} l(Y)
&= \sum_{i=p-K+1}^{p} \sum_{j=q-L+1}^{q} G_{i+K-1,j+L-1}  W_{i+K-p,j+L-q}.
\end{align}
Let us do a change of variable to write the above sum in terms of $W$ indices $k = i+K-p$ or $i=p-K+k$. Similarly, $l = j+L-q$ or $j=q-L+l$. This leads to,
\begin{align}
\pfrac{}{X_{p,q}} l(Y) &= \sum_{k=1}^{K} \sum_{l=1}^{L}  W_{k,l}G_{p+k-1,q+l-1}
\end{align}
The above is almost a convolution except that the $k$ and $l$ indices appear with a positive sign. To remedy that we use the mirror operator of $W$,
\begin{align}
\calM(W)_{k,l} &= W_{K+1-k,L+1-l} \text{ for all } k \in \{1, \dots, K\}, i \in \{ 1, \dots, L\}
\end{align}
Use this to write $\pfrac{l}{X_{p,q}}$ in terms of $\calM(G)$,
\begin{align}
\pfrac{}{X_{p,q}} l(Y) &= \sum_{k=1}^{K} \sum_{l=1}^{L}  \calM(W)_{K+1-k,L+1-l} G_{p+k-1,q+l-1}\\
&= \sum_{m=1}^{K} \sum_{n=1}^{L}  \calM(W)_{m,n} G_{p+K-m,q+L-n}
\end{align}

The  right hand side the convolution oeprator,
\begin{align}
\pfrac{}{X} l(Y) &= G * \calM(W)
\end{align}

Equivalently,
\begin{align}
\implies \pfrac{}{X} l(Y) &= \calM\left( \calM(G) * X \right)
\end{align}

In [252]:
def unpad(X, padding):
  return X[tuple(slice(s, -e) for s, e in padding)]

def mirror(X):
  return X[::-1, ::-1]

def convolve2d_vjp(X, W, dl__dY, padding=None):
  padding = handle_padding(padding)
  Xpadded = np.pad(X, padding)

  # Additional zero padding
  b_padding = [(s-1, s-1)  for s in W.shape]
  # The default padding needs to be reversed for dl__dY
  dl__dY_bpadded = np.pad(dl__dY, b_padding)

  dl__dW = ss.convolve2d(dl__dY, mirror(Xpadded), mode='valid')
  assert dl__dW.shape == W.shape

  # use the valid convolutions with custom padded X and W
  dl__dXp = ss.convolve2d(dl__dY_bpadded, mirror(W), mode='valid')
  assert dl__dXp.shape == Xpadded.shape
  dl__dX = unpad(dl__dXp, padding)

  return (dl__dX, dl__dW)

In [253]:
def numerical_convolve2d_vjp(X, W, dl__dY, h=1e-12, **kw):
  # We are going to perturb each pixel of X by a small amount h
  # and observe the corresponding change in Y
  dl__dX = np.empty_like(X)
  Y = t_convolve2d(X, W, **kw) # Y before any perturbation
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      Xp = X.copy()
      Xp[i, j] += h # perturbed image at pixel (i,j)
      Yp = t_convolve2d(Xp, W, **kw) # convolution result when perturbed
      # numerical derivative of image Y with respect to pixel X[i,j]
      dY__dXij = (Yp - Y) / h
      # numerical derivative of loss l with respect to pixel X[i,j]
      dl__dX[i, j] = (dl__dY * dY__dXij).sum()

  dl__dW = np.empty_like(W)
  for i in range(W.shape[0]):
    for j in range(W.shape[1]):
      Wp = W.copy()
      Wp[i, j] += h # perturbed kernel at pixel (i,j)
      Yp = t_convolve2d(X, Wp, **kw) # convolution result when perturbed
      # numerical derivative of image Y with respect to pixel X[i,j]
      dY__dWij = (Yp - Y) / h
      # numerical derivative of loss l with respect to pixel X[i,j]
      dl__dW[i, j] = (dl__dY * dY__dWij).sum()

  return dl__dX, dl__dW

random_dl__dY = np.ones(Y.shape)
nvjp_X, nvjp_W =numerical_convolve2d_vjp(X, W, random_dl__dY)
nvjp_X, nvjp_W

(array([[ 30.97966328,  60.02665032,  60.02665032],
        [ 54.00124792, 101.8634066 , 101.97709344],
        [ 53.88756108, 101.8634066 , 101.8634066 ]]),
 array([[45.98632586, 58.94662536],
        [54.17177817, 67.87104212]]))

In [254]:
vjp_X, vjp_W = convolve2d_vjp(X, W, random_dl__dY)
vjp_X, vjp_W

(array([[ 31.,  60.,  60.],
        [ 54., 102., 102.],
        [ 54., 102., 102.]]),
 array([[46., 59.],
        [54., 68.]]))

In [255]:
assert np.allclose(nvjp_X, nvjp_X, atol=1e-1, rtol=1e-2)
assert np.allclose(nvjp_W, nvjp_W, atol=1e-1, rtol=1e-2)

In [256]:
# Actually try a random one
random_dl__dY = np.random.rand(*Y.shape)
all([
    np.allclose(nvjp, cvjp, atol=1e-1, rtol=1e-2)
    for nvjp, cvjp in zip(
        numerical_convolve2d_vjp(X, W, random_dl__dY),
        convolve2d_vjp(X, W, random_dl__dY))
    ])

True

In [257]:
# All random values
X = np.random.rand(3,4)
W = np.random.rand(2,2)
Y = t_convolve2d(X, W)
random_dl__dY = np.random.rand(*Y.shape)
all([
    np.allclose(nvjp, cvjp, rtol=1e-2)
    for nvjp, cvjp in zip(
        numerical_convolve2d_vjp(X, W, random_dl__dY),
        convolve2d_vjp(X, W, random_dl__dY))
    ])

True

In [258]:
def test_convolve2d_vjp():
    # With random shapes
    W_shape = np.random.randint(1, 9, size=(2,))
    maxW = max(W_shape)
    X_shape = np.random.randint(maxW, 40, size=(2,))
    # All random values
    X = np.random.rand(*X_shape)
    W = np.random.rand(*W_shape)
    Y = t_convolve2d(X, W)
    random_dl__dY = np.random.rand(*Y.shape)
    return all([
    np.allclose(nvjp, cvjp, rtol=1e-2, atol=1e-3)
        for nvjp, cvjp in zip(
            numerical_convolve2d_vjp(X, W, random_dl__dY),
            convolve2d_vjp(X, W, random_dl__dY))
        ])
assert test_convolve2d_vjp()

In [259]:
# Run the test a hundred times just to make sure
for _ in range(100):
  assert test_convolve2d_vjp()